In [1]:
import json

def read_json_file(filepath):
    """
    Reads a JSON file and returns the data as a Python object.

    Args:
        filepath (str): The path to the JSON file.

    Returns:
        dict or list: The data from the JSON file, or None if an error occurs.
    """
    try:
        with open(filepath, 'r') as file:
            data = json.load(file)
        return data
    except FileNotFoundError:
        print(f"Error: File not found at {filepath}")
        return None
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in {filepath}")
        return None

In [7]:
import json

filename = "combined_smaller_sample_2"

json_data = read_json_file(f'output/{filename}.json')
print(json_data)

{'speakers': [{'speaker': 'SPEAKER_01', 'text': ' Yet, he is of all creatures the most formatively armed.', 'timestamp': [0.0, 4.52]}, {'speaker': 'SPEAKER_00', 'text': ' What then is the devilfish? It is the sea vampire.', 'timestamp': [4.52, 9.24]}, {'speaker': 'SPEAKER_02', 'text': ' To make sure this sad part of my story, we went the way of Hall sailors. The punch was made and I was made half drunk with it.', 'timestamp': [9.24, 18.04]}, {'speaker': 'SPEAKER_01', 'text': " At about two o'clock we heard the loud cry of sailho from a loft.", 'timestamp': [18.04, 23.4]}], 'asr': {'text': " Yet, he is of all creatures the most formatively armed. What then is the devilfish? It is the sea vampire. To make sure this sad part of my story, we went the way of Hall sailors. The punch was made and I was made half drunk with it. At about two o'clock we heard the loud cry of sailho from a loft.", 'chunks': [{'timestamp': [0.0, 4.52], 'text': ' Yet, he is of all creatures the most formatively arm

In [ ]:
segments = json_data["diarization"]
print("segments", segments)

# diarizer output may contain consecutive segments from the same speaker (e.g. {(0 -> 1, speaker_1), (1 -> 1.5, speaker_1), ...})
# we combine these segments to give overall timestamps for each speaker's turn (e.g. {(0 -> 1.5, speaker_1), ...})
new_segments = []
prev_segment = cur_segment = segments[0]

for i in range(1, len(segments)):
    cur_segment = segments[i]

    # check if we have changed speaker ("label")
    if cur_segment["label"] != prev_segment["label"] and i < len(segments):
        # add the start/end times for the super-segment to the new list
        new_segments.append(
            {
                "segment": {"start": prev_segment["segment"]["start"], "end": cur_segment["segment"]["start"]},
                "speaker": prev_segment["label"],
            }
        )
        prev_segment = segments[i]

# add the last segment(s) if there was no speaker change
new_segments.append(
    {
        "segment": {"start": prev_segment["segment"]["start"], "end": cur_segment["segment"]["end"]},
        "speaker": prev_segment["label"],
    }
)

print("new_segments", new_segments)

[{'segment': {'start': 0.45284375, 'end': 0.5372187500000001}, 'track': 'A', 'label': 'SPEAKER_01'}, {'segment': {'start': 0.5372187500000001, 'end': 8.02971875}, 'track': 'B', 'label': 'SPEAKER_00'}, {'segment': {'start': 9.160343750000003, 'end': 9.177218750000002}, 'track': 'C', 'label': 'SPEAKER_00'}, {'segment': {'start': 9.177218750000002, 'end': 17.36159375}, 'track': 'D', 'label': 'SPEAKER_02'}, {'segment': {'start': 17.98596875, 'end': 23.43659375}, 'track': 'E', 'label': 'SPEAKER_01'}]
[{'segment': {'start': 0.45284375, 'end': 0.5372187500000001}, 'speaker': 'SPEAKER_01'}, {'segment': {'start': 0.5372187500000001, 'end': 9.177218750000002}, 'speaker': 'SPEAKER_00'}, {'segment': {'start': 9.177218750000002, 'end': 17.98596875}, 'speaker': 'SPEAKER_02'}, {'segment': {'start': 17.98596875, 'end': 23.43659375}, 'speaker': 'SPEAKER_01'}]
